In [ ]:
#Copyright (c) 2025, Oracle and/or its affiliates.
#Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.

pip install azure-storage-blob

In [ ]:
#Copyright (c) 2025, Oracle and/or its affiliates.
#Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
import os
import oci
import io
import zipfile
import time
from azure.storage.blob import BlobServiceClient, BlobClient, ContentSettings

# OCI Configuration
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)

# Azure Configuration
azure_container_name = "<Azure Blob Storage>"

# OCI Object Storage Client
object_storage = oci.object_storage.ObjectStorageClient(config)

azure_connection_string = "DefaultEndpointsProtocol=https;AccountName=<..>;AccountKey=<..>;EndpointSuffix=core.windows.net"

# Azure Blob Service Client
blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
container_client = blob_service_client.get_container_client(azure_container_name)

In [ ]:
prefixes = [
    "fscmtopmodelam_finextractam_apbiccextractam",
    "fscmtopmodelam_finextractam_arbiccextractam",
    "fscmtopmodelam_finextractam_cebiccextractam",
    "fscmtopmodelam_finextractam_exmbiccextractam",
    "fscmtopmodelam_finextractam_fabiccextractam",
    "fscmtopmodelam_finextractam_funbiccextractam",
    "fscmtopmodelam_finextractam_glbiccextractam",
    "fscmtopmodelam_finextractam_iexbiccextractam",
    "fscmtopmodelam_finextractam_xlabiccextractam",
    "fscmtopmodelam_finextractam_xlebiccextractam",
    "fscmtopmodelam_finextractam_zxbiccextractam"
]

In [ ]:
import base64
def get_text_secret(secret_ocid):
    vault_client = oci.secrets.SecretsClient(config)
    try:
        secret_content = vault_client.get_secret_bundle(secret_ocid).data.secret_bundle_content.content
        decrypted_secret_content = base64.b64decode(secret_content).decode("utf-8")
    except Exception as ex:
        print("ERROR: failed to retrieve the secret content", ex, flush=True)
        raise
    return decrypted_secret_content

In [ ]:
import snowflake.connector

passowrd=get_text_secret("<oci vault secret>")

conn = snowflake.connector.connect(
    user='<>',
    password=passowrd,
    account='<>',
    warehouse='<>',
    database='<>',
    schema='<>'
)

cur = conn.cursor()

In [ ]:
def process_filename(filename):
    # Split the filename based on underscores
    parts = filename.split('_')
    
    # Ensure there are at least four underscores in the filename
    if len(parts) < 5:
        return "Invalid filename"
    
    # Join the parts after the fourth underscore
    new_filename = '_'.join(parts[4:])
    
    # Remove everything after the first hyphen
    new_filename = new_filename.split('-')[0]
    
    return new_filename+".csv"

In [ ]:
def checkLisOfObjects(path_file):
    for substring in prefixes:
        if substring in path_file:
            return True

In [ ]:
def uploadToAzure(file_contents, blob_client):
    with zipfile.ZipFile(io.BytesIO(file_contents)) as zip_ref:
    # List contents of the zip file
        for file_info in zip_ref.infolist():
            extracted_file_content = zip_ref.read(file_info.filename)
            blob_client.upload_blob(data=io.BytesIO(extracted_file_content), overwrite=True)

In [ ]:
source_bucket = '<OCI Bucket>'
source_namespace='<OCI Namespace>'
next_starts_with = None
# List objects in OCI Bucket
object_list = object_storage.list_objects(source_namespace, source_bucket)

# Copy objects from OCI to Azure Blob Storage
try:
    start_time = time.time()
    for obj in object_list.data.objects:
        object_name = obj.name
        if checkLisOfObjects(object_name):
            object_content = object_storage.get_object(source_namespace, source_bucket, object_name)
            blob_client = container_client.get_blob_client(process_filename(object_name))
            uploadToAzure(object_content.data.content, blob_client)
    if not next_starts_with:
        print('{"status":"200"}')
        end_time = time.time()
        elapsed_time= end_time - start_time
        print("Elapsed time: ", elapsed_time)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(f"Name: {blob.name}")

In [ ]:
bucket = container_client.list_blobs()
start_time = time.time()
for obj in bucket:
    object_name = obj.name[:-4]
    try:
        copy_table = f"COPY INTO {object_name} FROM '@<azure stage>/{object_name}.csv' FILE_FORMAT = (TYPE = 'csv', FIELD_DELIMITER=',', FIELD_OPTIONALLY_ENCLOSED_BY='\"', SKIP_HEADER = 1, error_on_column_count_mismatch=false)"
    except:
        copy_table = f"COPY INTO {object_name} FROM '@<azure stage>/{object_name}.csv' FILE_FORMAT = (TYPE = 'csv', FIELD_DELIMITER='|', FIELD_OPTIONALLY_ENCLOSED_BY='\"', SKIP_HEADER = 1,error_on_column_count_mismatch=false)"
    cur.execute(copy_table)
    conn.commit()
    print(copy_table)
end_time = time.time()
elapsed_time= end_time - start_time
print("Elapsed time: ", elapsed_time)
cur.close()
conn.close()